In [1]:
# Imports
load('prismatic_envelope.sage')
load('precision.py')
load('homology.sage')

### User-defined input

In [10]:
# The prime p
p=2

# The motivic weight i
i=16

# The power of the uniformizer n
n=2

# The residual degree f
# The present code only supports totally ramified extensions of Qp,
# i.e., where f=1
f=1

### Some precision calculations

In [11]:
# The calculated F-precision needed to compute at this weight
Fprec=n*i

# The target precision
target_precision=nygaard_exponent(p,i,n)

####################
# Precision losses #
####################

### From \delta
precision_loss_delta=math.floor(math.log(Fprec-1,p))

### From passing from OK to OK/pi^n
precision_loss_quotient=0
for q in range(p,i):
    precision_loss_quotient+=n*valuation(p,math.factorial(q))
    
### From lifting nabla to Nygaard
precision_loss_nygaard=n*math.floor(i*(i-1)/2)

### From computing can-phi on primitives
precision_loss_primitives=target_precision

### From renormalizing the Eisenstein polynomial
precision_loss_from_Eisenstein=1

total_precision=(target_precision+precision_loss_delta
                 +precision_loss_quotient
                 +precision_loss_nygaard
                 +precision_loss_primitives
                 +precision_loss_from_Eisenstein)

print("total p-adic precision is {}".format(total_precision))
print("Fprec is {}".format(Fprec))

# The coefficient ring W
if f==1:
    W=Zp(p,total_precision,type='capped-abs',print_mode='digits',show_prec=False)
else:
    W=Zq(p**f,total_precision,names='a',type='capped-abs',print_mode='series',show_prec=False)
    
# The Breuil-Kisin ring A
A.<z>=PowerSeriesRing(W,Fprec)

total p-adic precision is 309
Fprec is 32


### User-defined input: the Eisenstein polynomial

Note that in order for this to be created in a power series ring with the correct p-adic and F-adic precisions,
the elements p,i,n,f should be set above *before* defining the Eisenstein polynomial.

In [12]:
# The Eisenstein polynomial E
E=A(z+p)

### The main calculation

In [14]:
#%%capture
# Suppresses some Python warnings and SAGE variable injections

# The normalized Eisenstein polynomial
# The normalization is to bring the Eisenstein polynomial into the form E(0)=p
E=eisenstein_normalization(p,E)

# The syntomic matrices from new
%prun -s cumulative SyntomicComplex(p,i,n,E,total_precision,Fprec,debug=False)

<string>:21: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See http://trac.sagemath.org/14825 for details.


         67413984 function calls (67328979 primitive calls) in 174.884 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  174.884  174.884 {built-in method builtins.exec}
        1    0.001    0.001  174.884  174.884 <string>:14(__init__)
        1    0.100    0.100  174.852  174.852 <string>:784(syntomic_matrices)
        1    0.001    0.001  151.567  151.567 <string>:656(nablaP_matrix_OK)
      425    3.053    0.007  141.789    0.334 <string>:570(reduce)
       53    0.034    0.001  123.558    2.331 <string>:601(recursive_reduce)
   683255  105.481    0.000  110.791    0.000 multi_polynomial_element.py:260(_mul_)
        1    0.001    0.001   92.144   92.144 <string>:694(initialize_bk_factor)
        1    0.002    0.002   34.728   34.728 <string>:683(recursive_phiC_product)
        5    0.019    0.004   33.724    6.745 <string>:621(phi)
   168374    0.141    0.000   33.429    0.000 multi_polyno

### Assembling the syntomic complex and computing its cohomology